In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from functools import *
from GTF.Utilities import *

import networkx as nx
import pygsp as pg
import numpy as np
import random
import scipy as sp
from admm import admm

import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
from math import log
#from tqdm import tqdm, trange
from tqdm import tqdm_notebook
from tqdm import tnrange
import math

In [2]:
#create container of num_iter piecewise graph signals:
def noisy_pwc(num_iter, Gnx, num_seeds, noise_level):
    beta_0_list = []
    y_list = []
    path_lens = dict(nx.shortest_path_length(Gnx))
    for it in range(num_iter):
        beta_0 = gen_pwc_sig(Gnx,num_seeds,path_lens)
        y = beta_0.copy() + noise_level * np.random.rand(Gnx.number_of_nodes())
        beta_0_list.append(beta_0)
        y_list.append(y)
    
    beta_0_list = np.array(beta_0_list)
    y_list = np.array(y_list)
    return {'beta_0':beta_0_list,'y':y_list}

In [3]:
def controller(gamma, Gnx, k, rho, penalty_param,penalty_f,signals, num_iter, beta_init):
    mse_rs = 0.
#     l0diff_rs = 0.
    l0beta_rs = 0.
    suppTPR_rs = 0.
    suppFPR_rs = 0.
    betas = []
    for it in range(num_iter):
        beta_0 = signals['beta_0'][it]
        y = signals['y'][it]      
        if beta_init is not None:
            out = admm_mse2(gamma,rho,penalty_param,Gnx,beta_0,y,k,penalty_f,
                           beta_init=beta_init[it])
        else:
            out = admm_mse2(gamma,rho,penalty_param,Gnx,beta_0,y,k,penalty_f,
                           beta_init=None)          
        mse_rs += out['mse']
#         l0diff_rs += out['l0diff']
        l0beta_rs += out['l0beta']
        suppTPR_rs += out['suppTPR']
        suppFPR_rs += out['suppFPR']
        betas.append(out['beta'])
    
    output = {'avg_mse':mse_rs/num_iter, #'avg_l0diff':np.round(l0diff_rs/num_iter), 
            'avg_l0beta':np.around(l0beta_rs/num_iter, decimals=3), 'avg_suppTPR':np.around(suppTPR_rs/num_iter, decimals=3),
           'avg_suppFPR':np.around(suppFPR_rs/num_iter, decimals=3)}  
    if penalty_f == 'L1':
        output['beta'] = betas

    return output

In [4]:
G = pg.graphs.Minnesota(connect = True)
#G = pg.graphs.Grid2d(20,20)
#G = pg.graphs.ErdosRenyi(100,0.05)
#G = pg.graphs.Sensor(20,1)
#G.set_coordinates()
Gnx = nx.from_scipy_sparse_matrix(G.A.astype(float),edge_attribute = 'weight')
num_seeds = 4
path_lens = dict(nx.shortest_path_length(Gnx))
beta_0 = gen_pwc_sig(Gnx,num_seeds, path_lens) 
#beta_0 = beta_0 / np.linalg.norm(beta_0,2)


LOW NOISE

In [ ]:
noise_level = 0.05

y = beta_0 + noise_level * np.random.rand(G.N)
limits = [ min(y),  max(y)]
#example beta_0,y for current parameter setting

fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(15, 6))
pg.plotting.plot_signal(G,beta_0,limits = limits, plot_name = "Beta_0", ax = axes[0])
pg.plotting.plot_signal(G,y,limits = limits,plot_name = "y : noisy", ax = axes[1])


In [ ]:
num_iter = 1
signals = noisy_pwc(num_iter, Gnx, num_seeds, noise_level )




In [ ]:

num_pts = 100
log_gamma_sweep = np.linspace(-3,3,num_pts)
gamma_sweep = [10**(y) for y in log_gamma_sweep]

num_pts2 = 20
log_rhomult_sweep = np.linspace(-1,1,num_pts2)
rhomult_sweep = [10**(y) for y in log_rhomult_sweep]

In [ ]:
#find best for given gamma by sweeping over rho
import copy 
def sweep_rho(rm_sweep, gamma,Gnx,k,penalty_param,num_iter,penalty_f, signals , beta_init):
    min_mse = float('inf')
    
    for jj in tnrange(len(rm_sweep)):   
        #print jj
        wrapper_gtf = controller(gamma, Gnx=Gnx,k=k,rho=max(rm_sweep[jj]*gamma,1.0/penalty_param),penalty_param=penalty_param,
                        num_iter=num_iter,penalty_f = penalty_f,signals = signals, beta_init=beta_init)
        mse = wrapper_gtf['avg_mse']
        if(mse < min_mse):
            min_mse = mse 
            wrapper_min = copy.deepcopy(wrapper_gtf)    
    return wrapper_min
        

In [ ]:
k = 0
out_L1 = []
out_SCAD = []
out_MCP = []
for ii in  tnrange(num_pts):
    print ii
    wrapper_L1 = sweep_rho(rm_sweep = rhomult_sweep, gamma = gamma_sweep[ii],Gnx = Gnx,k = k
                           ,penalty_param = float('inf') ,num_iter = num_iter,penalty_f = "L1", signals = signals , beta_init = None)
    wrapper_SCAD = sweep_rho(rm_sweep = rhomult_sweep, gamma = gamma_sweep[ii],Gnx = Gnx,k = k
                           ,penalty_param = 3.7 ,num_iter = num_iter,penalty_f = "SCAD", signals = signals , beta_init = wrapper_L1['beta'])
    wrapper_MCP = sweep_rho(rm_sweep = rhomult_sweep, gamma = gamma_sweep[ii],Gnx = Gnx,k = k
                           ,penalty_param = 1.4 ,num_iter = num_iter,penalty_f = "MCP", signals = signals , beta_init = wrapper_L1['beta'])

    del wrapper_L1['beta']
    out_L1.append(wrapper_L1)
    out_SCAD.append(wrapper_SCAD)
    out_MCP.append(wrapper_MCP)
        
        
        
        
out_L1 = {k: [dic[k] for dic in out_L1] for k in out_L1[0] if k is not 'beta'}
out_SCAD = {k: [dic[k] for dic in out_SCAD] for k in out_SCAD[0]}
out_MCP = {k: [dic[k] for dic in out_MCP] for k in out_MCP[0]}


In [ ]:
ind = np.argsort(out_L1['avg_l0beta'])
x_L1 = np.array([out_L1['avg_l0beta'][i] for i in ind]).astype(int)
mse_L1 = np.array([out_L1['avg_mse'][i] for i in ind])
# l0diff_L1 = np.array([out_L1['avg_l0diff'][i] for i in ind])
ind2 = np.argsort(out_L1['avg_suppFPR'])
suppTPR_L1 = np.array([out_L1['avg_suppTPR'][i] for i in ind2])
suppFPR_L1 = np.array([out_L1['avg_suppFPR'][i] for i in ind2])

ind = np.argsort(out_SCAD['avg_l0beta'])
x_SCAD = np.array([out_SCAD['avg_l0beta'][i] for i in ind]).astype(int)
mse_SCAD = np.array([out_SCAD['avg_mse'][i] for i in ind])
# l0diff_SCAD = np.array([out_SCAD['avg_l0diff'][i] for i in ind])
ind2 = np.argsort(out_SCAD['avg_suppFPR'])
suppTPR_SCAD = np.array([out_SCAD['avg_suppTPR'][i] for i in ind2])
suppFPR_SCAD = np.array([out_SCAD['avg_suppFPR'][i] for i in ind2])

ind = np.argsort(out_MCP['avg_l0beta'])
x_MCP = np.array([out_MCP['avg_l0beta'][i] for i in ind]).astype(int)
mse_MCP = np.array([out_MCP['avg_mse'][i] for i in ind])
# l0diff_MCP = np.array([out_MCP['avg_l0diff'][i] for i in ind])
ind2 = np.argsort(out_MCP['avg_suppFPR'])
suppTPR_MCP = np.array([out_MCP['avg_suppTPR'][i] for i in ind2])
suppFPR_MCP = np.array([out_MCP['avg_suppFPR'][i] for i in ind2])

In [ ]:
#mse plot:
plt.figure()
plt.plot(x_L1, mse_L1, label='L1')
plt.plot(x_SCAD, mse_SCAD, label='SCAD')
plt.plot(x_MCP, mse_MCP, label='MCP')
plt.legend()

plt.figure()
plt.plot(suppFPR_L1, suppTPR_L1, 'o--', label='L1')
plt.plot(suppFPR_SCAD, suppTPR_SCAD, 'v--', label='SCAD')
plt.plot(suppFPR_MCP, suppTPR_MCP, '.--', label='MCP')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.title('Support Recovery ROC Curve')



In [ ]:
def smooth_curve(bins, x, y):
    mask = np.digitize(x, bins) # bins[i-1]< x <= bins[i]
    return np.array([y[mask==idx].mean() for idx in range(1,len(bins))])


In [ ]:
#binning (plot average value in each bin) to smooth out the curve
#print signals['y'].size
#print len(Gnx.edges)
#print x_L1
#print x_SCAD
#print x_MCP
num_bins = 20
bins_mse = np.linspace(min(x_L1.min(), x_SCAD.min(), x_MCP.min()), max(x_L1.max(), x_SCAD.max(), x_MCP.max()), num_bins)
#print bins
mse_L1_avg = smooth_curve(bins_mse, x_L1, mse_L1)
mse_SCAD_avg = smooth_curve(bins_mse, x_SCAD, mse_SCAD)
mse_MCP_avg = smooth_curve(bins_mse, x_MCP, mse_MCP)

plt.figure()
plt.plot(bins_mse[:-1], mse_L1_avg, 'o--', label='L1')
plt.plot(bins_mse[:-1], mse_SCAD_avg, 'v--', label='SCAD')
plt.plot(bins_mse[:-1], mse_MCP_avg, '.--',label='MCP')
plt.legend()
plt.title('mse')


In [ ]:
num_bins = 20
bins_fpr = np.linspace(min(suppFPR_L1.min(), suppFPR_SCAD.min(), suppFPR_MCP.min()), max(suppFPR_L1.max(), suppFPR_SCAD.max(), suppFPR_MCP.max()), num_bins)
#print bins
suppTPR_L1_s = smooth_curve(bins_fpr, suppFPR_L1, suppTPR_L1)
suppTPR_SCAD_s = smooth_curve(bins_fpr, suppFPR_SCAD, suppTPR_SCAD)
suppTPR_MCP_s = smooth_curve(bins_fpr, suppFPR_MCP, suppTPR_MCP)



plt.figure()
plt.plot(bins_fpr[:-1], suppTPR_L1_s, 'o--', label='L1')
plt.plot(bins_fpr[:-1], suppTPR_SCAD_s, 'v--', label='SCAD')
plt.plot(bins_fpr[:-1], suppTPR_MCP_s, '.--', label='MCP')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.title('Support Recovery ROC Curve')


HIGH NOISE


In [ ]:
noise_level= 1
y = beta_0 + noise_level * np.random.rand(G.N)
limits = [ min(y),  max(y)]


#example beta_0,y for current parameter setting
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(15, 6))
pg.plotting.plot_signal(G,beta_0,limits = limits, plot_name = "Beta_0", ax = axes[0])
pg.plotting.plot_signal(G,y,limits = limits,plot_name = "y : noisy", ax = axes[1])

num_iter = 1
num_pts = 1000
log_gamma_sweep = np.linspace(-4,8,num_pts)
log_gamma_sweep = [10**(y) for y in log_gamma_sweep]

In [ ]:
k = 0
out_L1 = []
out_SCAD = []
out_MCP = []
for ii in  tnrange(num_pts):
    print ii
    wrapper_L1 = controller(log_gamma_sweep[ii], Gnx=Gnx,k=k,rho=log_gamma_sweep[ii],penalty_param=0,#penalty_param,
                    num_iter=num_iter,penalty_f = "L1",signals = signals, beta_init=None)
    wrapper_SCAD = controller(log_gamma_sweep[ii], Gnx=Gnx,k=k,rho=max(log_gamma_sweep[ii], 1.0/3.7),penalty_param=3.7,#penalty_param,
                    num_iter=num_iter,penalty_f = "SCAD",signals = signals, beta_init=wrapper_L1['beta'])
    wrapper_MCP = controller(log_gamma_sweep[ii], Gnx=Gnx,k=k,rho=max(log_gamma_sweep[ii],1.0/1.4),penalty_param=1.4,#penalty_param,
                    num_iter=num_iter,penalty_f = "MCP",signals = signals, beta_init=wrapper_L1['beta'])
    del wrapper_L1['beta']
    out_L1.append(wrapper_L1)
    out_SCAD.append(wrapper_SCAD)
    out_MCP.append(wrapper_MCP)
out_L1 = {k: [dic[k] for dic in out_L1] for k in out_L1[0] if k is not 'beta'}
out_SCAD = {k: [dic[k] for dic in out_SCAD] for k in out_SCAD[0]}
out_MCP = {k: [dic[k] for dic in out_MCP] for k in out_MCP[0]}

In [ ]:
ind = np.argsort(out_L1['avg_l0beta'])
x_L1 = np.array([out_L1['avg_l0beta'][i] for i in ind]).astype(int)
mse_L1 = np.array([out_L1['avg_mse'][i] for i in ind])
# l0diff_L1 = np.array([out_L1['avg_l0diff'][i] for i in ind])
ind2 = np.argsort(out_L1['avg_suppFPR'])
suppTPR_L1 = np.array([out_L1['avg_suppTPR'][i] for i in ind2])
suppFPR_L1 = np.array([out_L1['avg_suppFPR'][i] for i in ind2])

ind = np.argsort(out_SCAD['avg_l0beta'])
x_SCAD = np.array([out_SCAD['avg_l0beta'][i] for i in ind]).astype(int)
mse_SCAD = np.array([out_SCAD['avg_mse'][i] for i in ind])
# l0diff_SCAD = np.array([out_SCAD['avg_l0diff'][i] for i in ind])
ind2 = np.argsort(out_SCAD['avg_suppFPR'])
suppTPR_SCAD = np.array([out_SCAD['avg_suppTPR'][i] for i in ind2])
suppFPR_SCAD = np.array([out_SCAD['avg_suppFPR'][i] for i in ind2])

ind = np.argsort(out_MCP['avg_l0beta'])
x_MCP = np.array([out_MCP['avg_l0beta'][i] for i in ind]).astype(int)
mse_MCP = np.array([out_MCP['avg_mse'][i] for i in ind])
# l0diff_MCP = np.array([out_MCP['avg_l0diff'][i] for i in ind])
ind2 = np.argsort(out_MCP['avg_suppFPR'])
suppTPR_MCP = np.array([out_MCP['avg_suppTPR'][i] for i in ind2])
suppFPR_MCP = np.array([out_MCP['avg_suppFPR'][i] for i in ind2])

In [ ]:
#mse plot:
plt.figure()
plt.plot(x_L1, mse_L1, label='L1')
plt.plot(x_SCAD, mse_SCAD, label='SCAD')
plt.plot(x_MCP, mse_MCP, label='MCP')
plt.legend()

plt.figure()
plt.plot(suppFPR_L1, suppTPR_L1, 'o--', label='L1')
plt.plot(suppFPR_SCAD, suppTPR_SCAD, 'v--', label='SCAD')
plt.plot(suppFPR_MCP, suppTPR_MCP, '.--', label='MCP')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.title('Support Recovery ROC Curve')



In [ ]:
#binning (plot average value in each bin) to smooth out the curve

bins_mse = np.linspace(min(x_L1.min(), x_SCAD.min(), x_MCP.min()), max(x_L1.max(), x_SCAD.max(), x_MCP.max()), 20)
#print bins
mse_L1_avg = smooth_curve(bins_mse, x_L1, mse_L1)
mse_SCAD_avg = smooth_curve(bins_mse, x_SCAD, mse_SCAD)
mse_MCP_avg = smooth_curve(bins_mse, x_MCP, mse_MCP)

plt.figure()
plt.plot(bins_mse[:-1], mse_L1_avg, 'o--', label='L1')
plt.plot(bins_mse[:-1], mse_SCAD_avg, 'v--', label='SCAD')
plt.plot(bins_mse[:-1], mse_MCP_avg, '.--',label='MCP')
plt.legend()
plt.title('mse')

In [ ]:
num_bins = 20
bins_fpr = np.linspace(min(suppFPR_L1.min(), suppFPR_SCAD.min(), suppFPR_MCP.min()), max(suppFPR_L1.max(), suppFPR_SCAD.max(), suppFPR_MCP.max()), num_bins)
#print bins
suppTPR_L1_s = smooth_curve(bins_fpr, suppFPR_L1, suppTPR_L1)
suppTPR_SCAD_s = smooth_curve(bins_fpr, suppFPR_SCAD, suppTPR_SCAD)
suppTPR_MCP_s = smooth_curve(bins_fpr, suppFPR_MCP, suppTPR_MCP)



plt.figure()
plt.plot(bins_fpr[:-1], suppTPR_L1_s, 'o--', label='L1')
plt.plot(bins_fpr[:-1], suppTPR_SCAD_s, 'v--', label='SCAD')
plt.plot(bins_fpr[:-1], suppTPR_MCP_s, '.--', label='MCP')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.title('Support Recovery ROC Curve')
